In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta

In [2]:
def generator():
    while True:
        yield

In [3]:
from datetime import timedelta, date

def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

start_dt = date(2005, 1, 1)
end_dt = date(2023, 9, 9)
dates = []
for dt in daterange(start_dt, end_dt):
    dates.append(dt.strftime("%Y-%m-%d"))
    
dates = dates[::-1]
len(dates)

6826

In [7]:
def crawl(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('h2', {'class': 'post-title'})
    a = [a_.find('a').get('href') for a_ in a if a_ and len(a_.find('a').get('href', ''))]
    return a

In [8]:
crawl('https://www.sunahsukasakura.com/search?updated-max=2023-07-10T12:20:00%2B08:00&max-results=10&by-date=false')

['https://www.sunahsukasakura.com/2023/07/gathering-makan-makan-dengan-fans.html',
 'https://www.sunahsukasakura.com/2023/07/apa-beza-warna-penutup-botol-air-mineral.html',
 'https://www.sunahsukasakura.com/2023/07/resepi-gulai-kerisik-daging-arwah-mak.html',
 'https://www.sunahsukasakura.com/2023/07/lirik-lagu-enchanted-by-taylor-swift.html',
 'https://www.sunahsukasakura.com/2023/07/ikan-selar-goreng-kicap-untuk-lunch.html',
 'https://www.sunahsukasakura.com/2023/07/ww-rose-caramel-latte.html',
 'https://www.sunahsukasakura.com/2023/07/gulai-kerisik-dan-asam-pedas-daging.html',
 'https://www.sunahsukasakura.com/2023/07/lawatan-ke-kandang-kambing-yang-aneh.html',
 'https://www.sunahsukasakura.com/2023/07/lirik-lagu-cinta-terakhir-by-aiman-sidek.html',
 'https://www.sunahsukasakura.com/2023/06/juadah-raya-aidil-adha-2023.html',
 'https://www.sunahsukasakura.com/2022/09/sayyidul-istighfar-penghulu-istighfar.html',
 'https://www.sunahsukasakura.com/2022/09/sarapan-mee-goreng-sis-lin.html

In [14]:
from concurrent.futures import ThreadPoolExecutor, as_completed

max_worker = 10

data = []
for i in tqdm(range(0, len(dates), max_worker)):
    b = dates[i: i + max_worker]
    urls = [f'https://www.sunahsukasakura.com/search?updated-max={t}T12:20:00%2B08:00&max-results=10&by-date=false' for t in b]
    done = False
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(crawl, url): url for url in urls}
        
        for future in as_completed(futures):
            r = future.result()
            data.extend(r)
            if not len(r):
                done = True
    
    if done:
        break

100%|█████████████████████████████████████████| 683/683 [18:15<00:00,  1.60s/it]


In [15]:
len(set(data))

4163

In [16]:
with open('sunahsukasakura.com-urls.json', 'w') as fopen:
    json.dump(list(set(data)), fopen)